**GEO6361, semaine 12 : Réseaux de neurones artificiels** (RNA)

Prenons des données immobilières réelles (échantillonnées dans les années 1970... ) et essayons de construire un modèle de prédiction de prix median.

#**Exemple d'apprentissage automatique par RNA**

## **1. Exploration des données**

### **1.1 Importons les modules et les données requises pour cette section**


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from tensorflow import keras

In [3]:
# https://www.kaggle.com/datasets/vikrishnan/boston-house-prices
# - CRIM per capita crime rate by town
# - ZN proportion of residential land zoned for lots over 25,000 sq.ft.
# - INDUS proportion of non-retail business acres per town
# - CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
# - NOX nitric oxides concentration (parts per 10 million)
# - RM average number of rooms per dwelling
# - AGE proportion of owner-occupied units built prior to 1940
# - DIS weighted distances to five Boston employment centres
# - RAD index of accessibility to radial highways
# - TAX full-value property-tax rate per $10,000
# - PTRATIO pupil-teacher ratio by town
# - B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
# - LSTAT % lower status of the population
# - MEDV Median value of owner-occupied homes in $1000's

noms_colonnes = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df = pd.read_csv("/content/housing.csv", header=None, delimiter=r"\s+", names = noms_colonnes)

df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [5]:
# Retirons la colonne B (voir https://github.com/scikit-learn/scikit-learn/issues/16155, https://towardsdatascience.com/things-you-didnt-know-about-the-boston-housing-dataset-2e87a6f960e8, https://medium.com/@docintangible/racist-data-destruction-113e3eff54a8)
df = df.('B', axis=1)

### **1.2 Explorons les données**

In [ ]:
df.head()

In [ ]:
# Vérifions le nombre de données manquantes avec isnull().sum():
df.

In [ ]:
# utilisons la méthode describe()
df.

In [ ]:
# Distribution des variables
df.hist(
    bins=50,
    figsize=(12,10)
    )

In [ ]:
# Corrélogramme
plt.figure(figsize=(12, 10))
sns.heatmap(
    df.corr(),
    annot=True,
    cmap='PiYG',
    center=0
    )
plt.show()

Préparation des listes X et y (que je mets dans des arrays NumPy)

In [12]:
# "X" est une array 2D contenant les valeurs des variables qui pourraient prédire le prix médians
X = df.drop(['MEDV'], axis = 1).values # Nous retirons la variable à prédire, et transformons le DF en array NumPy avec .values

In [ ]:
X

In [14]:
# "y" est une array 1D contenant les valeurs des prix médians
y = df['MEDV'].values

In [ ]:
y

## **2. Contruction du modèle**

### **2.1 Séparation des données d’entrainement et des données de test**

In [16]:
# On importe la méthode de SKlearn qui sépare aléatoirement notre jeu de données entre les données d'entrainement et les données de test
from sklearn.model_selection import train_test_split

In [18]:
# On sépare le jeu de données de manière à garder 2/3 des données pour l'entrainement du modèle, et 1/3 pour le test du modèle
X_train, X_test, y_train, y_test = (X, y, test_size=0.33)

### **2.2 Étape facultative (mais qui permet de faire converger plus rapidement le modèle) : normaliser les données**

In [19]:
# Nous créons une couche qui normalise les données selon les moyennes et écarts-types des variables (de manière à ce que la moyenne des variables soit toujours égale à 0 et l'écart type égal à 1) :
norm_layer = keras.layers.Normalization()
norm_layer.adapt(X)

### **2.3 Construction de l’architecture du réseau de neurones**

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Création d'un modèle vide de type séquentiel (dans lequel les calculs se propagnent de gauche à droite)



In [21]:
model = Sequential()

Ajouter la couche d'entrée normalisée

In [22]:
model.add(norm_layer)

Création des couches et de leurs fonctions d'activation : https://towardsdatascience.com/deep-learning-which-loss-and-activation-functions-should-i-use-ac02f1c56aa8, https://riptutorial.com/fr/machine-learning/example/31624/fonctions-d-activation

In [ ]:
# Ajout d'une première couche cachée
# Le nombre de neurones par couche est à déterminer en fonction du nombre de caractéristiques d'entrée, mais c'est également en partie un exercice d'essai/erreur : https://medium.com/geekculture/introduction-to-neural-network-2f8b8221fbd3
# Comme fonction d'activation, on utilise une des fonctions d'activation les plus simples et les plus communes : ReLU, qui est une unité de rectificateur linéraire (f(x)=0 quand x<0 et f(x)=x quand x>0.)
# Ici, on choisit d'avoir 9 neurones (les données d'entrée ont 12 variables)
model.add(Dense(9, input_dim=12, activation='relu'))

# Si on souhaite ajouter d'autres couches cachées (l'ordre des couches est déterminée par l'ordre de l'appel à la méthode "add")
model.add(Dense(9, activation='relu'))
# model.add(Dense(9,activation='relu'))

Création du neurone "résultat". On a besoin d'un seul résultat (température), on ne lui donne donc qu'un neurone

In [24]:
model.add(Dense(1, activation='linear'))

On compile le réseau avec un optimiseur et une méthode de calcul d'erreur

In [25]:
model.compile(
    optimizer='Adam', # Fonction de gradient de descente permettant d'optimiser, une itération après l'autre, les poids et biais du réseau (pour en savoir plus https://www.geeksforgeeks.org/intuition-of-adam-optimizer/)
    loss='mse' # "Mean Square Error" calcule l'erreur moyenne quadratique entre les données d'entrée et les résultats connus
    )

### **2.4 Optimisation du réseau (calcul des poids et des biais)**

In [ ]:
# Entraînement du modèle
model.fit(
    X_train, # Variables d'entrée
    y_train, # Résultats correspondant
    epochs=500, # Nombre d'époques
    verbose=1
    )

Affichage de l'historique de l'optimisation

In [ ]:
# Visualisons l'évolution de l'optimisation du modèle
df_loss = pd.DataFrame(model.history.history)
df_loss['epoch'] = model.history.epoch
# loss_df.plot()


plt.figure()
plt.xlabel('Epoch')
plt.ylabel('Erreur moyenne quadratique (en milliers de dollars^2)')
plt.plot(df_loss['epoch'], df_loss['loss'], label='Erreur données d\'entrainement')
plt.legend()
# plt.ylim([0,50])

## **3. Évaluation de la qualité du modèle**

Prédiction des prix médians avec les données de test (et non les données d'entrainement)

In [ ]:
# Utilisation du modèle pour prédire des valeurs de sortie
test_predictions = model.predict(X_test)

In [29]:
# On construit un tableau pour comparer les résultats de prédiction et les résultats effectivement observés
test_predictions = pd.Series(test_predictions.reshape(len(X_test,)))
pred_df = pd.DataFrame(y_test,columns=['Test y'])
pred_df = pd.concat([pred_df,test_predictions],axis=1)
pred_df.columns = ['Test y','Model predictions']

In [ ]:
pred_df

Graphique des données observées (Test y) par rapport aux données prédites par le modèle :

In [ ]:
sns.scatterplot(x='Test y',y='Model predictions',data=pred_df)

Importation de la fonction Sklearn pour calculer les erreurs

In [31]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

Nous voici avec une erreur moyenne absolue de :

In [32]:
mean_absolute_error(pred_df['Test y'],pred_df['Model predictions'])

2.534411252187398

À comparer avec la moyenne des prix (pour se faire une idée de la qualité du modèle)

In [33]:
df['MEDV'].mean()

22.532806324110677

Le modèle n'est donc pas TRÈS bon, mais 1) on a appris des choses, et 2) on peut modifier la structure du RNA pour essayer de l'améliorer :)

Pour utiliser le modèle avec de nouvelles données, on passe une liste des valeurs des 12 variables d'entrée (dans notre cas, 5 enregistrements contenant chacun 12 variables) :

In [ ]:
print(y_test[:5])

print(model.predict(X_test[:5]))

Pour sauvegarder notre modèle :

In [35]:
model.save('/content/modele_Boston.keras') # crée un fichier au format spécifique à Keras
# model.export('/content/modele_Boston') # crée un dossier, mais pas possible de le réimporter facilement dans Keras

Pour charger un modèle existant :

In [36]:
from tensorflow.keras.models import load_model
loaded_model = load_model('modele_Boston.keras')

In [ ]:
# liste_val = [........]
loaded_model.predict(X_test[:5])